# Covariance matrix prediction using DCC-GARCH

This program predicts the covariance matrix of n-securities using DCC-GARCH method. 
The formulas for this code have been obtained from two papers:

1)A test of  Covariance- Matrix Forecasting Methods by VALERIY ZAKAMULIN

2)Multivariate DCC-GARCH Model-With Various Error Distributions by Elisabeth Orskaug



Author:Saeed Rahman

Date-2/9/2017

In [43]:
import numpy as np
import pandas as pd
from arch import arch_model
import math

#for demostration purposes, random numbers are taken for the returns
stock_returns = pd.Series({'Google':np.random.normal(0, .2, 10),
                           'Apple':np.random.normal(0, .2, 10),
                           'Ebay':np.random.normal(0, .2, 10)})
# 2nd dataset to test
# stock_returns=pd.Series({'Google':[.1,.2,-.2,.4],
#                           'Apple':[.01,.04,.02,-.07],
#                           'Ebay':[.07,-.17,.3,.02],
#                         'Exxon':[.1,.2,.3,-.21]})

returns=np.array(list(stock_returns.values))
initial_numberof_returns=returns[1].size # size of any of the security

esubt=[]
q_list=[]


In [44]:
#Calculating the initial values
cov_matrix=np.cov(returns)
diag=cov_matrix.diagonal()
diag_matrix=np.diag(np.diag(cov_matrix))
garch11 = arch_model(diag, p=1, q=1)
res = garch11.fit(update_freq=10)
alpha=res.params[2]
beta=res.params[3]

esubt.append(np.matmul(np.linalg.inv(diag_matrix),returns))
q_list.append(np.ones((stock_returns.size,stock_returns.size)))

pred_cov=[]
new_returns=[]

Iteration:     10,   Func. Count:     68,   Neg. LLF: -9.604280165541626
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -9.60428165557
            Iterations: 11
            Function evaluations: 74
            Gradient evaluations: 11


In [45]:
#garch 1,1 forecasting
def simulate_GARCH(T, a0, a1, b1, sigma1):
    
    # Initialize our values
    X = np.ndarray(T)
    sigma = np.ndarray(T)
    sigma[0] = sigma1
    
    for t in range(1, T):
        # Draw the next x_t
        X[t - 1] = sigma[t - 1] * np.random.normal(0, 1)
        # Draw the next sigma_t
        sigma[t] = math.sqrt(a0 + b1 * sigma[t - 1]**2 + a1 * X[t - 1]**2)
        
    X[T - 1] = sigma[T - 1] * np.random.normal(0, 1)    
    
    return X, sigma

In [46]:
#DCC Garch
def dccGARCH(returns):    
    cov_matrix=np.cov(returns)
    diag=cov_matrix.diagonal()
    diag_matrix=np.diag(np.diag(cov_matrix))
    
    garch11 = arch_model(diag, p=1, q=1)
    res = garch11.fit(update_freq=10)
    alpha=res.params[2]
    beta=res.params[3]    
    
    esubt.append(np.matmul(np.linalg.inv(diag_matrix),returns))
    qdash=np.cov(esubt[-2])
    q_list.append(((1-alpha-beta)*qdash) + 
             (alpha*np.matmul(esubt[-2],esubt[-2].T)) + ((beta)*q_list[-1]))
    
    qstar=np.diag(np.diag(cov_matrix))
    qstar=np.sqrt(qstar)
    
    r=np.matmul(np.linalg.inv(qstar),q_list[-1],np.linalg.inv(qstar))
    
    pred_cov_temp=np.matmul(diag_matrix,r)
    pred_cov.append(np.matmul(pred_cov_temp,diag_matrix))

In [47]:
#The returns of each assets are modeled using Garch11, which in needed for the DCC-Garch
periods_to_predict=5 #setting the number of periods to predict
for n in returns:
    garch11 = arch_model(n, p=1, q=1)
    res = garch11.fit(update_freq=10)
    omega=res.params[1]
    alpha=res.params[2]
    beta=res.params[3]
    sig_ma=np.std(n)  # 1st assumption
    return_forecast,sigma_forecast=simulate_GARCH(periods_to_predict,omega,alpha,beta,sig_ma)
    n=np.append(n,return_forecast)
    new_returns.append(n)
    
print(new_returns) #returns of the asset in which the forecasted returns are added

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -7.12137323696
            Iterations: 9
            Function evaluations: 59
            Gradient evaluations: 9
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -1.47839438947
            Iterations: 9
            Function evaluations: 59
            Gradient evaluations: 9
Iteration:     10,   Func. Count:     66,   Neg. LLF: -1.2524323436582045
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -1.2524378751
            Iterations: 11
            Function evaluations: 72
            Gradient evaluations: 11
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.534050641115
            Iterations: 8
            Function evaluations: 54
            Gradient evaluations: 8
[array([ 0.01      ,  0.04      ,  0.02      , -0.07      ,  0.03615405,
       -0.02177667, -0.01652622

In [48]:
#Predcition of the covariance matrix by passing in the return values upto the prediction 
for i in range(initial_numberof_returns,new_returns[1].size): 
    dccGARCH(returns[:,0:i]) #Assumption 2, selecting the range of returns


Iteration:     10,   Func. Count:     68,   Neg. LLF: -9.604280165541626
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -9.60428165557
            Iterations: 11
            Function evaluations: 74
            Gradient evaluations: 11
Iteration:     10,   Func. Count:     68,   Neg. LLF: -9.604280165541626
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -9.60428165557
            Iterations: 11
            Function evaluations: 74
            Gradient evaluations: 11
Iteration:     10,   Func. Count:     68,   Neg. LLF: -9.604280165541626
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -9.60428165557
            Iterations: 11
            Function evaluations: 74
            Gradient evaluations: 11
Iteration:     10,   Func. Count:     68,   Neg. LLF: -9.604280165541626
Optimization terminated successfully.    (Exit mode 0)
            Current function val

In [49]:
pred_cov[-1] # The predicted covariance matrix of nth period

array([[ 0.0224541 , -0.00383043,  0.09627465, -0.07195966],
       [-0.00095718,  0.09348908,  0.03490703, -0.07865717],
       [ 0.02107526,  0.03057931,  0.10806054, -0.09257085],
       [-0.01390393, -0.06081917, -0.08170743,  0.12427946]])

Key assumptions that I am making: 

1)The initial sigma value for Garch11 model is taken as the standard deviation up till that point 

2)In modeling the covariance matrix using DCC-Garch, I have taken used the returns till that period from the start.